# Transformers for Text Summerization

As we discussed earlier transformers are great for many NLP tasks and text summerization is one such task.

In 2019, a team of researchers formed a new model based on the assertion every NLP problem can be represented as a Text-To-Text function(and if we in fact think about it, most of the tasks we expect a transformer to do falls in to such category). The idea was to train a transformer using transfer learning techniques during the training phase and finetune using text to text approach. 

This idea led to new performance level for transformer models and the model is called Text-To-Text Transfer Transformer or T5 for short. The researchers wanted the T5 model to be have task agnostic training process. To acheive that purpose they simply added a prefix to the sequences defining what need to be done. For example,

- 'translate English to German: \[sequence]'
- 'summerize: \[sequence]'

This way T5 models can get different tasks in single format. This led T5 to be used in many use cases with same parameters.

### T5 Architecture

As mentioned earlier T5 researchers was not interested in finding a new transformer architecture. Instead they were interested in making the transformer input agnostic. Therefore they used the original transformer architecture for their purpose. But they slightly changed few functionalities to match their need.

- T5 self attention is `order independent`. Means works on a set rather that a list of sequential tokens like previous models.

- Instead of using positional embedding technique of original transformer, this uses a relative postional embedding technique. Also usage of this embeddings are bit different compared to previous models.


> Read more descriptive details about the T5 model for better explanations.

***

### Text summerization with T5

We will use huggingface provided T5 model for this task.



In [1]:
import torch
import json

from transformers import T5ForConditionalGeneration, T5Config, T5Tokenizer

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [ ]:
device = torch.device('cpu') # can use gpu if needed